In [ ]:
####################################################### Set up paths #######################################################

ddbb    = ... # Database name. No spaces, no underscore (_)
mode    = 'dicom' # Image mode. Options: 'nifti', 'dicom'
model   = 'Mixed_model' # Model for OAR segmentation. Options: 'FR_model' (French rectum), 'Mixed_model' (French + Italian databases)
use_manual_OARs   = True # Option to use manual OAR segmentations instead of automatic ones. Options : False, True

####################################################### Set up paths #######################################################

In [ ]:
# Imports
import os
import numpy as np
import SimpleITK as sitk
import multiprocessing
from glob import glob
import pandas as pd
import warnings

from utils.utilities import check_if_exist
from utils.load_data import load_data
from utils.voi_extraction import run_voi_extraction
from utils.download_pretrained_weights import download_pretrained_weights
from utils.postprocessing import postprocessing_native, export2dicomRT
from utils.dm_computation import dm_computation

In [ ]:
print("-------------------------------------------------------------------------------------------------------")
print("1. LOADING ORIGINAL IMAGES...")
print("-------------------------------------------------------------------------------------------------------")

# Create directories for input data path and output data path
data_path = os.path.join(os.getcwd(), 'Input', ddbb)
check_if_exist(data_path, create=False) # Check that the path exists
out_path  = os.path.join(os.getcwd(), 'Output', ddbb)
check_if_exist(out_path)

# Load patients (DICOM or NIfTI images). The ddbb folder should contain a different folder for each case to process. In each
# case folder, the image scan should be saved in a sub-folder named "img", and the manual OAR segmentations - if available - in
# a sub-folder named "mOAR".
load_data(data_path, out_path, mode)

In [ ]:
print("-------------------------------------------------------------------------------------------------------")
print("2. VOI EXTRACTION...")
print("-------------------------------------------------------------------------------------------------------")

# VOI Extraction: Localization Network + Crop using the centroid of the coarse prosate segmentation. Check the result to ensure
# that appropriate VOI has been created. Sometimes some images are not well predicted and it's necessary to modify this VOI 
# manually to ensure that the OARs and urethra segmentations are accurate.

checkpoint_path = os.path.join(os.getcwd(), 'networks/LocalizationNet/LocalizationNet_weights.best.hdf5')
dir_ddbb     = os.path.join(out_path, 'imgs')
check_if_exist(os.path.join(out_path, 'Urethra', 'GT_VOI'))
if use_manual_OARs: 
    check_if_exist(os.path.join(out_path, 'mVOIs', 'imagesTs')) # Path to save generated VOIs
    check_if_exist(os.path.join(out_path, 'OARs', 'manual'))
else: check_if_exist(os.path.join(out_path, 'VOIs', 'imagesTs')) # Path to save generated VOIs

p = multiprocessing.Process(target=run_voi_extraction, args=(checkpoint_path, out_path, dir_ddbb, use_manual_OARs))
p.start()
p.join()

In [ ]:
print("-------------------------------------------------------------------------------------------------------")
print("3. OARs SEGMENTATION : Fine Segmentation Network")
print("-------------------------------------------------------------------------------------------------------")

# Environmnet Variables
main_dir  = os.path.join(os.getcwd(), 'networks/SegmentationNet/nnUNet')
# Can be set in the the .bashrc file or exported in the terminal:
os.environ['nnUNet_raw_data_base'] = os.path.join(main_dir,'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed']  = os.path.join(main_dir,'nnUNet_preprocessed')
os.environ['RESULTS_FOLDER']       = os.path.join(main_dir,'nnUNet_trained_models')
os.environ['MKL_SERVICE_FORCE_INTEL'] = "1"

if use_manual_OARs: path_imgs    = os.path.join(out_path, 'mVOIs', 'imagesTs')
else: path_imgs    = os.path.join(out_path, 'VOIs', 'imagesTs')
dir_ddbb_OARs = os.path.join(out_path, 'OARs', model)

# Predict OAR segmentations (rectum, bladder, prostate, seminal vesicles)
if model=='FR_model':
    download_pretrained_weights(task_id=112)
    !nnUNet_predict -i {path_imgs} -o {dir_ddbb_OARs} -t 112 -tr nnUNetTrainerV2 -ctr nnUNetTrainerV2CascadeFullRes -m 3d_fullres -p nnUNetPlansv2.1 --disable_tta

if model=='Mixed_model':
    download_pretrained_weights(task_id=113)
    !nnUNet_predict -i {path_imgs} -o {dir_ddbb_OARs} -t 113 -tr nnUNetTrainerV2 -ctr nnUNetTrainerV2CascadeFullRes -m 3d_fullres -p nnUNetPlansv2.1 --disable_tta

# Post-processing and exportation
for i, file_OARs in enumerate(sorted(glob(dir_ddbb_OARs + r'/*.nii.gz'))):
    idx = file_OARs.split('/')[-1].split('_')[-1].split('.')[0]
    print(file_OARs)
    out_OARs  = sitk.ReadImage(file_OARs)
    out_VOI   = sitk.ReadImage(os.path.join(path_imgs, ddbb+'_'+idx+'_0000.nii.gz'))

    # 3.1 Post-processing to Native space
    print("3.1 OARs SEGMENTATION POST-PROCESSING TO NATIVE SPACE -------------------------------------------------")
    postprocessing_native(out_OARs, out_path, str(idx),'OARs', ddbb, use_manual_OARs=use_manual_OARs)
    postprocessing_native(out_VOI, out_path, str(idx),'VOI', ddbb, use_manual_OARs=use_manual_OARs)

    # 3.2 Exportation to Dicom-RT
    print("3.2 OARs SEGMENTATION EXPORTATION TO DICOM-RT ---------------------------------------------------------")
    img_file  = os.path.join(out_path, 'Native', str(idx), ddbb+'_'+str(idx)+'_VOI.nii.gz')
    seg_file = os.path.join(out_path, 'Native', str(idx), ddbb+'_'+str(idx)+'_OARs.nii.gz')
    save_path = os.path.join(out_path, 'DICOM', str(idx))
    try: export2dicomRT(img_file, seg_file, save_path, 'OARs_DLUS')
    except: print('----------------------------- Error exporting to DICOM ---------------------------')

In [ ]:
print("-------------------------------------------------------------------------------------------------------")
print("4. DISTANCE MAP COMPUTATION...")
print("-------------------------------------------------------------------------------------------------------")

# Create paths to save the distance maps
if use_manual_OARs: check_if_exist(os.path.join(out_path, 'mDistanceMaps', 'imagesTs'))
else: check_if_exist(os.path.join(out_path, 'DistanceMaps', 'imagesTs'))
    
# Bladder & Prostate Segmentation and DM Computation
dm_computation(dir_ddbb, out_path, model, use_manual_OARs)

In [ ]:
print("-------------------------------------------------------------------------------------------------------")
print("5. URETHRA SEGMENTATION")
print("-------------------------------------------------------------------------------------------------------")

if use_manual_OARs:
    path_dm = os.path.join(out_path, 'mDistanceMaps', 'imagesTs')
    dir_ddbb_uretra = os.path.join(out_path, 'Urethra', 'manualDLUS')
else:
    path_dm = os.path.join(out_path, 'DistanceMaps', 'imagesTs')
    dir_ddbb_uretra = os.path.join(out_path, 'Urethra', 'DLUS')

download_pretrained_weights(task_id=108)
!nnUNet_predict -i {path_dm} -o {dir_ddbb_uretra} -t 108 -tr nnUNetTrainerV2 -ctr nnUNetTrainerV2CascadeFullRes -m 3d_fullres -p nnUNetPlansv2.1 --disable_tta

# Post-processing and exportation
for i, file_out_urethra in enumerate(sorted(glob(dir_ddbb_uretra + r'/*.nii.gz'))):
    idx = file_out_urethra.split('/')[-1].split('_')[-1].split('.')[0]
    print(file_out_urethra)
    out_urethra = sitk.ReadImage(file_out_urethra)

    # 5.1 Post-processing to Native space
    print("5.1 URETHRA SEGMENTATION POST-PROCESSING TO NATIVE SPACE ----------------------------------------------")
    if use_manual_OARs:    postprocessing_native(out_urethra, out_path, str(idx),'m_urethra', ddbb, use_manual_OARs=use_manual_OARs)
    else: postprocessing_native(out_urethra, out_path, str(idx),'urethra', ddbb, use_manual_OARs=use_manual_OARs)
    
    # 5.2 Exportation to Dicom-RT
    print("5.2 URETHRA SEGMENTATION EXPORTATION TO DICOM-RT ------------------------------------------------------")
    img_file  = os.path.join(out_path, 'Native', str(idx), ddbb+'_'+str(idx)+'_VOI.nii.gz')
    seg_file = os.path.join(out_path, 'Native', str(idx), ddbb+'_'+str(idx)+'_urethra.nii.gz')
    save_path = os.path.join(out_path, 'Native', 'DICOM', str(idx))
    try: export2dicomRT(img_file, seg_file, save_path, 'urethra_DLUS')
    except: print('----------------------------- Error exporting to DICOM ---------------------------')